# Modify the code to clear the offset


basically using the transaction of GCP(地理控制点)并使用 sar_grid_latitude、sar_grid_longitude 和 sar_grid_height 作为其地理坐标。

AMSR暂时不生成，存在坐标系不匹配

In [1]:

import os
import numpy as np
import netCDF4
from osgeo import gdal, osr

import csv
import json


In [2]:

# Paths to folders
input_folder = r'E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\subNC'
tiff_sar_folder = os.path.join(input_folder, 'tiff_SAR')
#tiff_btemp_folder = os.path.join(input_folder, 'tiff_btemp')
tiff_polygon_folder = os.path.join(input_folder, 'tiff_polygon_icechart')
csv_output_folder = os.path.join(input_folder, 'csv_output')

# Create folders if they don't exist
os.makedirs(tiff_sar_folder, exist_ok=True)
#os.makedirs(tiff_btemp_folder, exist_ok=True)
os.makedirs(tiff_polygon_folder, exist_ok=True)
os.makedirs(csv_output_folder, exist_ok=True)


In [3]:
# Iterate through netCDF files
for filename in os.listdir(input_folder):
    if filename.endswith('.nc'):
        nc_file = os.path.join(input_folder, filename)
        ncf = netCDF4.Dataset(nc_file)
    
        lines = np.array(ncf.variables.get('sar_grid_line')) 
        samps = np.array(ncf.variables.get('sar_grid_sample')) 
        lats = np.array(ncf.variables.get('sar_grid_latitude')) 
        lons = np.array(ncf.variables.get('sar_grid_longitude')) 
        hgts = np.array(ncf.variables.get('sar_grid_height'))
        spr_gcp = osr.SpatialReference() 
        spr_gcp.ImportFromEPSG(4326) 
        gcps=() 
        for i in range(len(lines)): 
            x, y, z, pix, lin = lons[i],lats[i],hgts[i],samps[i],lines[i] 
            gcps = gcps + (gdal.GCP(x, y, z, pix, lin, '', str(i)),) 
            
            
            
        # Process polygon data
        if 'polygon_icechart' in ncf.variables and 'polygon_codes' in ncf.variables:
            polygon_data = np.nan_to_num(ncf.variables['polygon_icechart'][:])
            polygon_codes = ncf.variables['polygon_codes'][:]
            rows, cols = polygon_data.shape

            polygon_tiff_path = os.path.join(tiff_polygon_folder, f"{os.path.splitext(filename)[0]}_Polygon.tif")
            gtiff = gdal.GetDriverByName('GTiff').Create(polygon_tiff_path, cols, rows, 1, gdal.GDT_Float32)
            gtiff.SetGCPs(gcps, spr_gcp.ExportToWkt()) 

            band = gtiff.GetRasterBand(1)
            #print(polygon_data)
            band.WriteArray(polygon_data)
            band.SetDescription('polygon_icechart')
            gtiff = None

            csv_path = os.path.join(csv_output_folder, f"{os.path.splitext(filename)[0]}_PolygonCodes.csv")
            with open(csv_path, mode='w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['Index', 'Code'])
                writer.writerows(enumerate(polygon_codes))
            print(f"Saved Polygon Codes CSV: {csv_path}")
        
        # Extract SAR-related variables
        sar_variables = ['sar_secondary', 'sar_primary', 'nersc_sar_primary', 'nersc_sar_secondary']
        sar_bands = []
        band_names = []

        # Extract SAR coordinates
        sar_lon = ncf.variables['sar_grid_longitude'][:]
        sar_lat = ncf.variables['sar_grid_latitude'][:]
        
        for var in sar_variables:
            if var in ncf.variables:
                sar_bands.append(np.nan_to_num(ncf.variables[var][:]))
                band_names.append(var)

       # 生成 GeoTIFF
        if sar_bands:
            rows, cols = sar_bands[0].shape
            sar_tiff_path = os.path.join(tiff_sar_folder, f"{os.path.splitext(filename)[0]}_SAR.tif")
            gtiff = gdal.GetDriverByName('GTiff').Create(sar_tiff_path, cols, rows, len(sar_bands), gdal.GDT_Float32)

            # 定义空间参考（EPSG 4326）
            spatial_ref = osr.SpatialReference()
            spatial_ref.ImportFromEPSG(4326)

            # 生成 GCP（地理控制点）
            gcps=() 
            for i in range(len(lines)): 
                x, y, z, pix, lin = lons[i],lats[i],hgts[i],samps[i],lines[i] 
                gcps = gcps + (gdal.GCP(x, y, z, pix, lin, '', str(i)),) 

            # 设置 GCP
            gtiff.SetGCPs(gcps, spatial_ref.ExportToWkt())

            # 写入多个波段
            for idx, (band_data, band_name) in enumerate(zip(sar_bands, band_names), start=1):
                band = gtiff.GetRasterBand(idx)
                band.WriteArray(band_data)
                band.SetDescription(band_name)

            # 关闭 TIFF
            gtiff = None
            print(f"Saved TIFF: {sar_tiff_path}")

        

        ncf.close()

print("Processing complete.")

Saved Polygon Codes CSV: E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\subNC\csv_output\20180903T082725_S1B_AMSR2_IcechartGreenlandNorthEast_PolygonCodes.csv
Saved TIFF: E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\subNC\tiff_SAR\20180903T082725_S1B_AMSR2_IcechartGreenlandNorthEast_SAR.tif
Saved Polygon Codes CSV: E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\subNC\csv_output\20180910T081814_S1B_AMSR2_IcechartGreenlandNorthEast_PolygonCodes.csv
Saved TIFF: E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\subNC\tiff_SAR\20180910T081814_S1B_AMSR2_IcechartGreenlandNorthEast_SAR.tif
Saved Polygon Codes CSV: E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\subNC\csv_output\20180910T081914_S1B_AMSR2_IcechartGreenlandNorthEast_PolygonCodes.csv
Saved TIFF: E:\Sea Ice Classification\AI4ArcticASIP Sea ice Dataset version2\2018.910\subNC\tiff_SAR\20180910T081914_S